In [1]:
%env VECLIB_MAXIMUM_THREADS=1
%env MKL_NUM_THREADS=1
%env NUMEXPR_NUM_THREADS=1
%env OMP_NUM_THREADS=1

!make clean

import numpy as np
#np.show_config()

env: VECLIB_MAXIMUM_THREADS=1
env: MKL_NUM_THREADS=1
env: NUMEXPR_NUM_THREADS=1
env: OMP_NUM_THREADS=1
rm -rf *.o *.dSYM/ 01_skip_access 02_locality 03_matrix_matrix


# Cache optimization

1. How cache works and its importance to performance
2. Stride analysis
3. Tiling

# Memory hierarchy

To show how cache works and its importance to performance, see table below:

| Type | Latency (cycles) |
| -- | -- |
| CPU registers | 0 |
| L1 cache | 4 |
| L2 cache | 10 |
| L3 cache | 50 |
| Main memory | 200 |
| Disk / drive | 100,000 |

(Excerpt from Figure 6.23 in CS:APP (3/e).)

Remarks:
* Disk (or SSD) is slow.  Don't do I/O unless absolutely necessary.
* CPU instruction is 100x faster than memory access.
* Cache hides the memory latency.

Nothing is fast without cache.

## How cache works

Assume we have a main memory of 64 bytes, and a cache of 8 bytes.

| Access # | Decimal addr | Binary addr | Hit or miss | Cache block (binary) |
| -- | -- | -- | -- | -- |
| 1 | 22 | 10110 | miss (cold) | 110 |
| 2 | 26 | 11010 | miss (cold) | 010 |
| 3 | 22 | 10110 | hit | 110 |
| 4 | 26 | 11010 | hit | 010 |
| 5 | 16 | 10000 | miss (cold) | 000 |
| 6 | 18 | 10010 | miss (conflict) | 010 |
| 7 | 26 | 11010 | miss (conflict) | 010 |

This is a direct-map cache.  To reduce conflict misses, we may use multi-way set associativity.  2-, 4-, or 8-way set associative cache is commonly used.  Full associativity is too expensive in circuit implementation.

# Memory accessing speed is determined by cache block (line) size

A cache block usually contains more than one byte or word.  In x86, the block (line) size is 64 bytes.  That is, when loading data from main memory to cache, it's done once a block, rather than byte by byte.

```cpp
// Sequential.
for (int i=0; i<nelem; ++i) { arr[i] = i; }
sw.lap();
for (int i=0; i<nelem; ++i) { arr[i] *= 3; }
elapsed = sw.lap();

// Skipping 2.
for (int i=0; i<nelem; ++i) { arr[i] = i; }
sw.lap();
for (int i=0; i<nelem; i+=2) { arr[i] *= 3; }
elapsed = sw.lap();

// ... 4, 8, 16, ... 1024.
```

In [2]:
# Show how main memory (dram) access dominates runtime.
!make 01_skip_access ; ./01_skip_access

g++  -std=c++17 -O3 -g -m64 -I/opt/intel/mkl/include /opt/intel/mkl/lib/libmkl_intel_lp64.a /opt/intel/mkl/lib/libmkl_sequential.a /opt/intel/mkl/lib/libmkl_core.a -lpthread -lm -ldl  -o 01_skip_access 01_skip_access.cpp
Sequential takes: 0.0591595 sec

Skipping 2 takes: 0.0666328 sec
Skipping 4 takes: 0.0642925 sec
Skipping 8 takes: 0.0616888 sec
Skipping 16 takes: 0.0660376 sec

Skipping 32 takes: 0.0452228 sec
Skipping 64 takes: 0.0290477 sec
Skipping 128 takes: 0.0172844 sec
Skipping 256 takes: 0.0114255 sec
Skipping 512 takes: 0.00685178 sec
Skipping 1024 takes: 0.00233585 sec


# Locality

While coding we usually don't have a lot of time to do detailed cache analysis.  Instead, we keep in mind that the code runs faster when it's more compact.  This is the concept of locality.

There are two kinds of locality: temporal and spatial.  Temporal locality means that a fixed address is reused in the near future.  Spatial locality means that the addresses close to the current address is reused in the near future.  The better locality, of either kind, improves the performance.  And the cache hierarchy is why locality works.

To take advantage of locality, programmers analyze by using "strides".  A stride is the number of indexes to elements to slide when accessing the data in arrays.  The most basic striding is sequential access, or the 1-stride.  Similarly, we may have n-strides.  The larger the stride is, the worse the locality.

Recall that x86 uses 64-byte cache blocks, and a double-precision floating point takes 8 bytes.

## Demonstrate by matrix population

To demonstrate how the data layout, i.e., majoring or striding, affects runtime, we use an example of populating $1024 \times 1024 \times 64$ integer elements as a matrix.  The following shapes are benchmarked (total number of elements remains unchanged):

* $(1024\times1024\times64) \times 1$, i.e., one-dimension
* $(1024\times1024\times32) \times 2$
* $(1024\times1024\times16) \times 4$
* $(1024\times1024\times8) \times 8$
* $(1024\times1024\times4) \times 16$
* $(1024\times1024\times2) \times 32$
* $(1024\times1024) \times 64$
* $(1024\times512) \times 128$
* $(1024\times256) \times 256$
* $(1024\times128) \times 512$
* $(1024\times64) \times 1024$
* $(1024\times32) \times (1024\times2)$
* $(1024\times16) \times (1024\times4)$
* $(1024\times8) \times (1024\times8)$

We populate the matrices along two axes.  First we iterate over the last index (row-major):

```cpp
// Populate by last index.
for (size_t i=0; i<nrow; ++i) // the i-th row
{
    for (size_t j=0; j<ncol; ++j) // the j-th column
    {
        buffer[i*ncol + j] = i*ncol + j;
    }
}
```

Then iterate over the first index (column-major):

```cpp
// Populate by first index.
for (size_t j=0; j<ncol; ++j) // the j-th column
{
    for (size_t i=0; i<nrow; ++i) // the i-th row
    {
        buffer[i*ncol + j] = i*ncol + j;
    }
}
```

We will see the speed is very different.

In [3]:
# Show how striding affects population runtime.
!make 02_locality ; ./02_locality

g++  -std=c++17 -O3 -g -m64 -I/opt/intel/mkl/include /opt/intel/mkl/lib/libmkl_intel_lp64.a /opt/intel/mkl/lib/libmkl_sequential.a /opt/intel/mkl/lib/libmkl_core.a -lpthread -lm -ldl  -o 02_locality 02_locality.cpp
# of elements: 67108864 = 67108864 x 1
populate double flatly takes: 0.059887 sec
populate double along last axis takes: 0.0986658 sec
populate double along first axis takes: 0.049492 sec
ratio: 0.501612

# of elements: 67108864 = 33554432 x 2
populate double flatly takes: 0.0491339 sec
populate double along last axis takes: 0.0771682 sec
populate double along first axis takes: 0.101107 sec
ratio: 1.31022

# of elements: 67108864 = 16777216 x 4
populate double flatly takes: 0.0508462 sec
populate double along last axis takes: 0.0543871 sec
populate double along first axis takes: 0.20397 sec
ratio: 3.75034

# of elements: 67108864 = 8388608 x 8
populate double flatly takes: 0.0560276 sec
populate double along last axis takes: 0.0534111 sec
populate double along first axis tak

While writing programs, it's much easier to know the stride than analyzing the cache behavior.  The latter, in many scenarios, is prohibitively difficult.

Since we know the cache line is 64 byte wide, we expect the cache performance may significantly reduce when the stride is around that value (16 int elements).  As shown in the above benchmark.

### Condition the memory and the cache before benchmarking

```cpp
// Prepopulation to cancel the effect of overcommit or delayed allocation.
for (size_t i=0; i<nelem; ++i) { buffer[i] = nelem-i; }
```

## Array majoring in numpy

Array majoring is directly related to locality.  The difference in the performance of matrix-vector multiplication is show for row- and column-majoring arrays.

In [4]:
%%time
dim = 10000
float_rmajor = np.arange(dim*dim, dtype='float64').reshape((dim,dim))
float_cmajor = float_rmajor.T.copy().T
vec = np.arange(dim, dtype='float64')

CPU times: user 1.14 s, sys: 368 ms, total: 1.51 s
Wall time: 1.51 s


In [5]:
%%time
res_float_rmajor = np.dot(float_rmajor, vec)

CPU times: user 64 ms, sys: 1.22 ms, total: 65.2 ms
Wall time: 63.5 ms


In [6]:
%%time
res_float_cmajor = np.dot(float_cmajor, vec)

CPU times: user 130 ms, sys: 1.21 ms, total: 131 ms
Wall time: 129 ms


### Integer matrix-vector multiplication

In [7]:
%%time
dim = 10000
int_rmajor = np.arange(dim*dim, dtype='int64').reshape((dim,dim))
int_cmajor = int_rmajor.T.copy().T
vec = np.arange(dim, dtype='int64')

CPU times: user 1.12 s, sys: 388 ms, total: 1.51 s
Wall time: 1.51 s


In [8]:
%%time
res_int_rmajor = np.dot(int_rmajor, vec)

CPU times: user 70.6 ms, sys: 968 µs, total: 71.5 ms
Wall time: 70.3 ms


In [9]:
%%time
res_int_cmajor = np.dot(int_cmajor, vec)

CPU times: user 794 ms, sys: 1.49 ms, total: 796 ms
Wall time: 794 ms


The performance difference of integer arrays is much larger than floating-point arrays.  Note that `double` and `int64` both take 8 bytes.  Reason: LAPACK / MKL / vecLib.

For the same reason, the floating-point multiplication is slightly faster than the integer.

# Tiling

This is a naive implementation of matrix-matrix multiplication:

```cpp
for (size_t i=0; i<mat1.nrow(); ++i)
{
    for (size_t k=0; k<mat2.ncol(); ++k)
    {
        double v = 0;
        for (size_t j=0; j<mat1.ncol(); ++j)
        {
            v += mat1(i,j) * mat2(j,k);
        }
        ret(i,k) = v;
    }
}
```

The matrices are row-major.  The stride for the second matrix is `ncol2`, so it doesn't have good locality.  This naive implementation is clear, but the performance is bad.

Matrix-matrix multiplication is one of the most important problems for numerical calculation, and there are many techniques available for making it fast.  Most if not all of them are about hiding the memory access latency.  Tiling is a basic technique that delivers impressive speed-up by reordering the calculation and making it cache-friendly.

In [10]:
# Show how tiling improves runtime performance.
!make 03_matrix_matrix ; ./03_matrix_matrix

g++  -std=c++17 -O3 -g -m64 -I/opt/intel/mkl/include /opt/intel/mkl/lib/libmkl_intel_lp64.a /opt/intel/mkl/lib/libmkl_sequential.a /opt/intel/mkl/lib/libmkl_core.a -lpthread -lm -ldl  -o 03_matrix_matrix 03_matrix_matrix.cpp
Timing mkl: 0.044965 second, 1.07374 Gflo, 23.8795 Gflops
Timing indirect: 2.85728 second, 1.07374 Gflo, 0.375791 Gflops
Timing direct: 3.0651 second, 1.07374 Gflo, 0.350312 Gflops
Timing tiled 32: 1.1393 second, 1.07374 Gflo, 0.942457 Gflops
Timing tiled 64: 0.454891 second, 1.07374 Gflo, 2.36044 Gflops
Timing tiled 128: 0.773399 second, 1.07374 Gflo, 1.38834 Gflops
Timing tiled 256: 0.838338 second, 1.07374 Gflo, 1.2808 Gflops
Timing tiled 512: 0.76896 second, 1.07374 Gflo, 1.39636 Gflops
Timing tiled 1024: 0.939641 second, 1.07374 Gflo, 1.14272 Gflops


# Exercises

1. Consult the data sheet of one x86 CPU and one Arm CPU.  Make a table for the line size of each of the cache levels, and compare the difference between the two CPUs.
2. Write a program that uses tiling to speed up matrix-matrix multiplication, and do not require the matrix dimension to be multiples of the tile size. 

# References

* Gallery of Processor Cache Effects: http://igoro.com/archive/gallery-of-processor-cache-effects/
* Lecture Notes of Applications of Parallel Computers by David Bindel: https://www.cs.cornell.edu/~bindel/class/cs5220-s10/slides/lec03.pdf